In [1]:
!pip install -q transformers torch
!pip install datasets

!pip uninstall transformers
!pip install --no-cache-dir transformers sentencepiece
!pip install rouge_score
!pip install evaluate

     |████████████████████████████████| 5.5 MB 4.7 MB/s 
     |████████████████████████████████| 182 kB 84.7 MB/s 
     |████████████████████████████████| 7.6 MB 74.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.4 MB/s 
     |████████████████████████████████| 212 kB 74.4 MB/s 
     |████████████████████████████████| 115 kB 83.1 MB/s 
     |████████████████████████████████| 127 kB 68.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Found existing installation: transformers 4.24.0
Uninstalling transformers-4.24.0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.7/dist-packages/transformers-4.24.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/transformers/*
Proceed (y/n)? y
  Successfully uninstalled transformers-4.24.0
Lookin

In [2]:
from transformers import TFAutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from datasets import load_dataset
from datasets import concatenate_datasets, DatasetDict

from datasets import load_dataset
import pandas as pd
import torch
from transformers.tokenization_utils_base import BatchEncoding
from typing import List, Tuple
from tqdm import tqdm
from transformers import AdamW, get_linear_schedule_with_warmup
from datasets import load_metric
import tensorflow as tf
import os
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
from nltk.tokenize import sent_tokenize
import nltk
import evaluate
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [44]:
df = load_dataset('csv', data_files = {'train': 'sample_data/cnn_train_class0.csv',
                                                              'validation':'sample_data/cnn_val_class0.csv',
                                                              'test':'sample_data/cnn_test_class0.csv'})
df

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'article', 'highlights', 'class'],
        num_rows: 107197
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'article', 'highlights', 'class'],
        num_rows: 3937
    })
    test: Dataset({
        features: ['Unnamed: 0', 'article', 'highlights', 'class'],
        num_rows: 3498
    })
})

In [53]:
ds_train = df['train']

In [54]:
ds_val = df['validation']

In [55]:
ds_test = df['test']

In [56]:
ds_train

Dataset({
    features: ['Unnamed: 0', 'article', 'highlights', 'class'],
    num_rows: 107197
})

In [57]:
ds_val

Dataset({
    features: ['Unnamed: 0', 'article', 'highlights', 'class'],
    num_rows: 3937
})

In [58]:
ds_test

Dataset({
    features: ['Unnamed: 0', 'article', 'highlights', 'class'],
    num_rows: 3498
})

In [59]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [60]:
model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


Downloading:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [62]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [63]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token: 
Add token as git credential? (Y/n) y
Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [64]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [65]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["article"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["highlights"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [66]:
tokenized_train = ds_train.map(preprocess_function, batched=True)
tokenized_val =ds_val.map(preprocess_function, batched=True)
tokenized_test =ds_test.map(preprocess_function, batched=True)


  0%|          | 0/108 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [67]:
tokenized_train=tokenized_train.remove_columns(ds_train.column_names)
tokenized_test=tokenized_test.remove_columns(ds_test.column_names)
tokenized_val=tokenized_val.remove_columns(ds_val.column_names)

features = [tokenized_train[i] for i in range(2)]

In [68]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_train,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8)
  
tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_val,
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8)
  

num_train_epochs = 8
num_train_steps = len(tf_train_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01)


model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [69]:
callback = PushToHubCallback(
    output_dir=f"{model_name}-finetuned-test", tokenizer=tokenizer
)

model.fit(
    tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback], epochs=8
)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/repository.py:729: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  FutureWarning,
Cloning https://huggingface.co/ririying/mt5-small-finetuned-test into local empty directory.


Epoch 1/8
13399/13399 [==============================] - ETA: 0s - loss: 3.5536

Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


13399/13399 [==============================] - 1412s 104ms/step - loss: 3.5536 - val_loss: 2.1181
Epoch 2/8
13399/13399 [==============================] - 1376s 103ms/step - loss: 2.4769 - val_loss: 1.9296
Epoch 3/8
13399/13399 [==============================] - 1360s 101ms/step - loss: 2.2865 - val_loss: 1.8569
Epoch 4/8
13399/13399 [==============================] - 1359s 101ms/step - loss: 2.1928 - val_loss: 1.8241
Epoch 5/8
13399/13399 [==============================] - 1354s 101ms/step - loss: 2.1344 - val_loss: 1.8022
Epoch 6/8
13399/13399 [==============================] - 1356s 101ms/step - loss: 2.0953 - val_loss: 1.7880
Epoch 7/8
13399/13399 [==============================] - 1357s 101ms/step - loss: 2.0671 - val_loss: 1.7811
Epoch 8/8
13399/13399 [==============================] - 1356s 101ms/step - loss: 2.0505 - val_loss: 1.7733


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file tf_model.h5:   0%|          | 3.33k/2.07G [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/ririying/mt5-small-finetuned-test
   48bbebc..c0e43f0  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/ririying/mt5-small-finetuned-test
   48bbebc..c0e43f0  main -> main



In [71]:
from tqdm import tqdm
import numpy as np

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=320
)

tf_generate_dataset = model.prepare_tf_dataset(
    tokenized_val,
    collate_fn=generation_data_collator,
    shuffle=False,
    batch_size=8,
    drop_remainder=True,
)

In [72]:
@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=32,
    )

In [73]:
all_preds = []
all_labels = []
for batch, labels in tqdm(tf_generate_dataset):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    all_preds.extend(decoded_preds)
    all_labels.extend(decoded_labels)

100%|██████████| 492/492 [01:03<00:00,  7.74it/s]


In [74]:
rouge_score = evaluate.load("rouge")
result = rouge_score.compute(
    predictions=decoded_preds, references=decoded_labels, use_stemmer=True
)
result = {key: value* 100 for key, value in result.items()}
{k: round(v, 4) for k, v in result.items()}

{'rouge1': 32.0663, 'rouge2': 15.2027, 'rougeL': 24.4691, 'rougeLsum': 29.0423}

#### Rouge score for test

In [75]:
from tqdm import tqdm
import numpy as np

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=320
)

tf_generate_dataset = model.prepare_tf_dataset(
    tokenized_test,
    collate_fn=generation_data_collator,
    shuffle=False,
    batch_size=8,
    drop_remainder=True,
)

In [ ]:
@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=32,
    )

In [76]:
all_preds = []
all_labels = []
for batch, labels in tqdm(tf_generate_dataset):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    all_preds.extend(decoded_preds)
    all_labels.extend(decoded_labels)

100%|██████████| 437/437 [00:37<00:00, 11.75it/s]


In [77]:
rouge_score = evaluate.load("rouge")
result = rouge_score.compute(
    predictions=decoded_preds, references=decoded_labels, use_stemmer=True
)
result = {key: value* 100 for key, value in result.items()}
{k: round(v, 4) for k, v in result.items()}

{'rouge1': 37.7853, 'rouge2': 22.9882, 'rougeL': 34.352, 'rougeLsum': 35.4361}

In [85]:
import pandas as pd
df_class0_test = pd.DataFrame(all_preds)

In [79]:
df_test = pd.DataFrame(df['test'])

In [81]:
df_test.head(), len(df_test)

(   Unnamed: 0                                            article  \
 0           0  (CNN)The Palestinian Authority officially beca...   
 1           2  (CNN)If you've been following the news lately,...   
 2           3  (CNN)Five Americans who were monitored for thr...   
 3           6  (CNN)Governments around the world are using th...   
 4           8  (CNN)Filipinos are being warned to be on guard...   
 
                                           highlights  class  
 0  Membership gives the ICC jurisdiction over all...      0  
 1  Mohammad Javad Zarif has spent more time with ...      0  
 2  17 Americans were exposed to the Ebola virus w...      0  
 3  Amnesty's annual death penalty report catalogs...      0  
 4  Once a super typhoon, Maysak is now a tropical...      0  , 3498)

In [83]:
df_test_new = df_test.iloc[:3498, 1:]
df_test_new.head(), len(df_test_new)

(                                             article  \
 0  (CNN)The Palestinian Authority officially beca...   
 1  (CNN)If you've been following the news lately,...   
 2  (CNN)Five Americans who were monitored for thr...   
 3  (CNN)Governments around the world are using th...   
 4  (CNN)Filipinos are being warned to be on guard...   
 
                                           highlights  class  
 0  Membership gives the ICC jurisdiction over all...      0  
 1  Mohammad Javad Zarif has spent more time with ...      0  
 2  17 Americans were exposed to the Ebola virus w...      0  
 3  Amnesty's annual death penalty report catalogs...      0  
 4  Once a super typhoon, Maysak is now a tropical...      0  , 3498)

In [86]:
df_test_new['summary'] = df_class0_test
df_test_new.head()

,article,highlights,class,summary
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,0,Palestinian Authority officially became 123rd ...
1,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,0,Mohammad Javad Zarif is a U.S. Secretary of St...
2,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,0,Five Americans were monitored for three weeks ...
3,(CNN)Governments around the world are using th...,Amnesty's annual death penalty report catalogs...,0,Amnesty International says governments using t...
4,(CNN)Filipinos are being warned to be on guard...,"Once a super typhoon, Maysak is now a tropical...",0,Tropical storm Maysak approached Asian island ...


In [87]:
df_test_new.to_csv('test_class0_final.csv')

In [90]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [91]:
!cp test_class0_final.csv "drive/My Drive/580project"

In [92]:
from transformers import pipeline

hub_model_id = "ririying/mt5-small-finetuned-test"
summarizer = pipeline("summarization", model=hub_model_id)

Downloading:   0%|          | 0.00/745 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at ririying/mt5-small-finetuned-test.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/390 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

In [94]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token: 
Add token as git credential? (Y/n) y
Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [96]:
model.push_to_hub("ririying/my-finetuned-mt5-class0")

In [ ]:
# use the fine-tune model
#from transformers import pipeline

#hub_model_id = 'SiriRRR/mt5-small-finetuned-test'
#summarizer = pipeline("summarization", model=hub_model_id)

Downloading:   0%|          | 0.00/745 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at SiriRRR/mt5-small-finetuned-test.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


Downloading:   0%|          | 0.00/390 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/74.0 [00:00<?, ?B/s]